# OECM Benchmark Data Pipeline

The Benchmark data pipelines organize and assemble benchmark data needed for the ITR tool.  This pipeline supports the OECM Benchmark version 2 (published 4 May 2022).


### Environment variables and dot-env

The following cell looks for a "dot-env" file in some standard locations,
and loads its contents into `os.environ`.

In [1]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import numpy as np
import pandas as pd
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc
# import python_pachyderm

Define Environment and Execution Variables

In [2]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

In [3]:
import io
import json
import itertools

In [4]:
# See data-platform-demo/pint-demo.ipynb for quantify/dequantify functions

import warnings  # needed until quantile behaves better with Pint quantities in arrays
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType
from openscm_units import unit_registry

# openscm_units doesn't make it easy to set preprocessors.  This is one way to do it.
unit_registry.preprocessors=[
     # lambda s1: s1.replace('passenger km', 'passenger_km'),
     lambda s2: s2.replace('BoE', 'boe'),
]

PintType.ureg = unit_registry
ureg = unit_registry
set_application_registry(ureg)
Q_ = ureg.Quantity
PA_ = PintArray

ureg.define("CO2e = CO2 = CO2eq = CO2_eq")
ureg.define("Fe = [iron] = Steel")
ureg.define("iron = Fe")
ureg.define("Al = [aluminum] = Aluminum")
ureg.define("aluminum = Al")
ureg.define("Cement = [cement]")
ureg.define("cement = Cement")

# For reports that use 10,000 t instead of 1e3 or 1e6                                                                                                                                                                                                        
ureg.define('myria- = 10000')

# These are for later
ureg.define('fraction = [] = frac')
ureg.define('percent = 1e-2 frac = pct = percentage')
ureg.define('ppm = 1e-6 fraction')

ureg.define("USD = [currency]")
ureg.define("EUR = nan USD")
ureg.define("JPY = nan USD")

ureg.define("btu = Btu")
ureg.define("mmbtu = 1e6 btu")
# ureg.define("boe = 5.712 GJ")                                                                                                                                                                                                                              
ureg.define("boe = 6.1178632 GJ")
ureg.define("mboe = 1e3 boe")
ureg.define("mmboe = 1e6 boe")

# Transportation activity

ureg.define("vehicle = [vehicle] = v")
ureg.define("passenger = [passenger] = p = pass")
ureg.define("vkm = vehicle * kilometer")
ureg.define("pkm = passenger * kilometer")
ureg.define("tkm = tonne * kilometer")

ureg.define('hundred = 1e2')
ureg.define('thousand = 1e3')
ureg.define('million = 1e6')
ureg.define('billion = 1e9')
ureg.define('trillion = 1e12')
ureg.define('quadrillion = 1e15')

### S3 and boto3

In [5]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

### Connecting to Trino with sqlalchemy

In the context of the Data Vault, this pipeline operates with full visibiilty into all the data it prepares for the ITR tool.  When the data is output, it is labeled so that the Data Vault can enforce its data management access rules.

In [6]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
dera_schema = 'sandbox'
dera_prefix = 'dera_'
gleif_schema = 'sandbox'
rmi_schema = 'sandbox'
iso3166_schema = 'sandbox'
essd_schema = 'sandbox'
essd_prefix = 'essd_'
demo_schema = 'demo_dv'

engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog)

using connect string: trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev


### Definitions and dictionaries for reading from / writing to the outside world

In [7]:
transport_elements = ['Subsector', 'Total CO2 Emissions', 'Emission Intensity', 'Energy Intensity']
bldg_elements = ['Parameter', 'Residential Buildings', 'Commercial Buildings']

benchmark_years = pd.Series(name='Production', index=pd.Index(list(range(2019,2051))), dtype='float64')
benchmark_years.index.name = 'Year'

# Maps Sector (really Sub-Sector) to Sheet data
oecm_dict = {
    # Subsector: Parameter / Subsector tag; Sheet; Aggregates as; Aggregates to; CO2 label; Production Units; Intensity Units
    'Materials / Steel': ['Parameter', 'Steel', 'Materials / Steel', 'Annual production volume- Iron & Steel Industry', 'Total CO2 equivalent', 'Mt Steel', 't CO2e/(t Steel)'],
    'Power Utilities': ['Subsector', 'Utilities', 'Power Utilities', 'Total public power generation (incl. CHP, excluding auto producers, losses)', 'Total CO2 equivalent', 'TWh', 't CO2e/MWh'],
    'Gas Utilities': ['Subsector', 'Utilities', 'Gas Utilities', 'Total Energy transport & distribution (gas, synthetic fuels & hydrogen)', 'Total CO2 equivalent', 'PJ', 't CO2e/MWh'],
    'Utilities': ['Subsector', 'Utilities', 'Utilities', 'Total Energy Production (power + gas/fuels)', 'Total CO2 equivalent', 'PJ', 't CO2e/MWh'],
    'Energy Industry': ['Subsector', 'Energy', 'Energy Industry', 'Total FINAL ENERGY RE-FT production', 'Total CO2 equivalent', 'PJ', 't CO2e/GJ'],
    'Road: LDV / Passenger Transport': ['Subsector', 'Transport_UNPRI', 'Road Transport', 'Road Transport (excluding vehicle manufacturing)', '', 'km', 'g CO2e/pkm'],
    'Road: Trucks / Freight Transport': ['Subsector', 'Transport_UNPRI', 'Road Transport', 'Road Transport (excluding vehicle manufacturing)', '', 'km', 'g CO2e/tkm'],
    'Aluminium Industry': ['Parameter', 'Alu', 'Aluminium Industry', 'Annual production volume- aluminium Industry', 'Total CO2 equivalent', 'Mt Aluminum', 't CO2e/(t Aluminum)'],
    'Materials / Cement': ['Parameter', 'Cement', 'Materials / Cement', 'Cement - production volume in mega tonnes per year', 'Total CO2 equivalent', 'Mt Cement', 't CO2e/(t Cement)'],
    'Buildings': ['Parameter', 'Buildings', 'Buildings', 'Construction: Residential and Commercial Building - Economic value', 'Total CO2 equivalent', 'billion m**2', 't CO2e/(thousand m**2)'],
    'Chemical Industry': ['Parameter', 'Chemical Industry', 'Chemical Industry', 'Total Chemical Industry', 'Total CO2 equivalent', 'billion USD', 'kg CO2e/USD'],
    'Textile & Leather': ['Parameter', 'Tex & Lea', 'Textile & Leather', 'Total Textile & Leather', 'Total CO2 equivalent', 'billion USD', 'kg CO2e/USD'],
}

# From OECM (Sub-)Sector name to ITR Sector Name.  Keys MUST BE UNIQUE
itr_dict = {
    'Materials / Steel': 'Steel',
    'Power Utilities': 'Electricity Utilities',
    'Gas Utilities': 'Gas Utilities',
    'Utilities': 'Utilities',
    'Energy Industry': 'Oil & Gas',
    'Road: LDV / Passenger Transport': 'Autos',
    'Road: Trucks / Freight Transport': 'Trucking',
    'Aluminium Industry': 'Aluminum',
    'Materials / Cement': 'Cement',
    'Buildings': 'Buildings',
    'Chemical Industry': 'Chemicals',
    'Textile & Leather': 'Textiles'
}

### Interpolation Function

Production is CAGR-based; Emissions are CAGR-based if the ratio fo start/finish <= 2.

When start/finish gets too high, the curve gets a pronounced drop in the first year

When finish is zero, the curve can only approach is asymptotically, which is also problematic.
Instead, use linear interpolation when it's time to drive the curve down to zero

In [8]:
# Interpolate missing benchmark values for Production and Emissions, then compute Emissions Intensities (EI)

def interpolate_benchmark(df, ei_unit, first_year=2019, last_year=2050):
    # Interpolate all missing Production and Scope emissions, except Scope 3 remains zero until we change benchmarks

    i = first_year
    while i < last_year:
        idx1 = i # .Production.first_valid_index()
        idx2 = df[df.index>i].Production.first_valid_index()
        if idx2 is None:
            break

        nth_root = 1/(idx2-idx1)
        for col in ['Production', 'S1', 'S2', 'S1S2', 'S3', 'S1S2S3']:
            if df.loc[idx2, col] == 0 or (df.loc[idx1, col]/df.loc[idx2, col]).m > 2:
                # print(f"Linear: {df.loc[idx1, col].m}/{df.loc[idx2, col].m}")
                # Linear interpolation
                delta = (df.loc[idx2, col] - df.loc[idx1, col]) / (idx2-idx1)
                for j in range(idx1, idx2):
                    df.loc[j+1, col] = df.loc[j, col] + delta
            else:
                # print(f"CAGR: {df.loc[idx1, col].m}/{df.loc[idx2, col].m}")
                # CAGR interpolation
                multiplier = ((df.loc[idx2, col] / df.loc[idx1, col])**nth_root).m
                for j in range(idx1, idx2):
                    df.loc[j+1, col] = df.loc[j, col] * multiplier
        i = idx2
    df['EI_S1S2'] = (df.S1S2 / df.Production).astype(f"pint[{ei_unit}]")
    df['EI_S3'] = (df.S3 / df.Production).astype(f"pint[{ei_unit}]")
    df['EI_S1S2S3'] = (df.S1S2S3 / df.Production).astype(f"pint[{ei_unit}]")

    # By convention, the d_ column is zero at the start of the series.
    # Subsequent values multiply the previous quantity by the present d_ number to get the present quanity
    df["d_Production"] = [0] + [m.m-1 for m in (df.Production.values[1:]/df.Production.values[:-1]).tolist()]

    return df

### Principle processing function

Start with dataframe containing "messy" data from Spreadsheet, then clean it up to a standard format

In [9]:
def process_sector_benchmark (sector_benchmark, region, subsector, sector_elements):
    s = sector_benchmark.iloc[:, 1]
    sector = sector_elements[2]
    # Transport_UNPRI doesn't have 'Total CO2 equivalent' in its scope strings...
    df_elements = [sector_elements[0],sector_elements[3],
                   ' '.join([f"{sector} - Scope 1:", sector_elements[4]]).rstrip(),
                   ' '.join([f"{sector} - Scope 2:", sector_elements[4]]).rstrip(),
                   ' '.join([f"{sector} - Scope 3:", sector_elements[4]]).rstrip()]
    
    # Hand-adjust the rows and columns we'll be processing.  A few sectors are unique in their shape/data.
    # Some sheets have extra years of data, which pushes 2050 to the right.  We allocate a generous number
    # of columns so that we capture 2050, and then we drop the columns we don't need, either from middle or the right
    sheet = sector_elements[1]
    if sheet == 'Transport_UNPRI':
        display(sector_benchmark.iloc[s.loc[s.isin(df_elements).fillna(False)].index, 1:14])
    if sheet == 'Chemical Industry':
        df = sector_benchmark.iloc[s.loc[s.isin(df_elements).fillna(False)].index, 1:14][[True]*2 + [False]*6 + [True]*3]
    elif sheet == 'Tex & Lea':
        df = sector_benchmark.iloc[s.loc[s.isin(df_elements).fillna(False)].index, 1:14][[True]*2 + [False]*4 + [True]*3]
    else:
        df = sector_benchmark.iloc[s.loc[s.isin(df_elements).fillna(False)].index, 1:14][[True]*2 + [False]*3 + [True]*3]
    while df.iloc[0, -1] != '2050':
        df = df.drop(columns=df.columns[-1])
    
    # Column 'D' is either empty or contains notes to self...drop in either case
    df = df.drop(columns=df.columns[2])
    # Drop empty columns and transpose so that years are in rows
    df = df.dropna(how='all', axis=1).T

    # Now ready to build the DataFrame...
    df.columns = ['Year', 'Production', 'S1', 'S2', 'S3']
    df.S3 = df.S3.fillna(0)
    units = df.iloc[1, 1:].map(lambda x: x[1:-1].split('/')[0].replace('Mt CO2 equiv.', 'Mt CO2e'), na_action='ignore')
    units.replace("bn $ GDP", "billion USD")
    units.Production = sector_elements[5]
    df = df.iloc[2:].astype({'Year':'int', 'Production':'float', 'S1':'float', 'S2':'float', 'S3':'float'}).set_index('Year')

    # Fix Transport by proportionalizing results to Autos only.  When and how to handle Freight?
    # Note that we have three main transport types: Aviation, Shipping, Road, and two main carriage types: Passenger and Freight
    # For now, we just handled Road Transport
    if sheet == 'Transport_UNPRI':
        # Need to proportionalize total sector emissions vs. passenger-only and then feed back into total
        s = pd.concat([sector_benchmark.iloc[:8, 1],sector_benchmark.iloc[87:, 1]])
        road = sector_benchmark.iloc[s.loc[s.isin(transport_elements).fillna(False)].index, 1:14]
        while road.iloc[0, -1] != '2050':
            road = road.drop(columns=road.columns[-1])
        if subsector == 'Road: LDV / Passenger Transport':
            road = road.dropna(how='all', axis=1)[1:4].T
        else:
            road = road.dropna(how='all', axis=1)[4:7].T
        road.columns = road.iloc[0]
        road_units = road.iloc[1].map(lambda x: x[1:-1].split('/')[0].replace('Mt CO2 equiv.', 'Mt CO2e'), na_action='ignore')
        road_km = 'pkm' if subsector=='Road: LDV / Passenger Transport' else 'tkm'
        for unit in road_units.index:
            if 'Intensity' in unit:
                road_units[unit] = f"{road_units[unit]} / {road_km}"
        units.Production = (ureg(road_units['Total CO2 Emissions']) / ureg(road_units['Emission Intensity'])).to(f"giga{road_km}").u
        road = road.iloc[2:].astype('float64')
        road.index = df.index
        # Slice out old data columns so that everything starts at 2019
        df = df.drop([2017,2018], errors='ignore')
        road = road.drop([2017,2018], errors='ignore')
        df = pd.concat([df, road], axis=1)
        df.Production = df.apply(lambda x: Q_(x['Total CO2 Emissions'], road_units['Total CO2 Emissions'])
                                 / Q_(x['Emission Intensity'], road_units['Emission Intensity']) if x['Emission Intensity'] else np.nan,
                                 axis=1).fillna(method='ffill')
        scopes = ['S1', 'S2', 'S3']
        total_co2 = df[scopes].sum(axis=1)
        for scope in scopes:
            df[scope] = (df[scope] * df['Total CO2 Emissions'] / total_co2).replace(np.nan, 0)
        df = df.drop(columns=transport_elements[1:])
    elif sheet == 'Buildings':
        bldgs = sector_benchmark.iloc[s.loc[s.isin(bldg_elements).fillna(False)].index, 1:14]
        while bldgs.iloc[0, -1] != '2050':
            bldgs = bldgs.drop(columns=bldgs.columns[-1])
        bldgs = bldgs.dropna(how='all', axis=1).T
        bldgs.columns = bldgs.iloc[0]
        bldgs_units = bldgs.iloc[1].map(lambda x: x[1:-1])
        units.Production = ureg('billions m**2').u
        bldgs = bldgs.iloc[2:].astype('float64')
        bldgs.index = df.index
        df.Production = bldgs.sum(axis=1)

    # Now insert all the missing years we need to interpolate
    df = pd.DataFrame(benchmark_years).combine_first(df)
    # Change type at the end, as the addition of np.nan values can mess with the dtype (making it dtype 'object')
    for col in df.columns:
        df[col] = df[col].astype(f"pint[{units[col]}]")
    df.insert(0, "Sector", subsector)
    df.insert(0, "Region", region)
    df['S1S2'] = df.S1 + df.S2
    df['S1S2S3'] = df.S1 + df.S2 + df.S3
    return interpolate_benchmark(df, sector_elements[6])

### Construct JSON benchmark structures

1.  Load Regional Workbook
2.  Process each Sector in the Workbook
3.  Convert resulting dataframe to dictionary structure
4.  Merge each Region/Sector dictionary into main benchmark dictionary

Note that we use linear interpolation when the overall interpolation is more than a 2:1 ratio start to finish
CAGR gets wonky both as the endpoint approaches zero (ratio becomes infinite); but it's also funky when slope is steep (though not infinitely steep)

In [11]:
production_bm = {
    "benchmark_temperature": "1.5 delta_degC",
    "benchmark_global_budget": "396 Gt CO2",
    "is_AFOLU_included": False,
}

ei_bm = {
    "benchmark_temperature": "1.5 delta_degC",
    "benchmark_global_budget": "396 Gt CO2",
    "is_AFOLU_included": False,
}

region_dict = {'Global':'OECM_Global_2022_04_22_Results',
               'Europe':'OECM_OECD_Europe_2022_04_22_results',
               'North America':'OECM_OECD_North_America_2022_04_22_results_0'}

def merge_bm_dicts(main, new):
    for scope in new.keys():
        if not main.get(scope):
            main[scope] = { 'benchmarks': [] }
        main[scope]['benchmarks'].append(new[scope]['benchmarks'][0])

for subsector, sector_elements in oecm_dict.items():
    sheet = sector_elements[1]
    ei_unit = sector_elements[6]
    for region, filename in region_dict.items():
        df = pd.read_excel(os.environ.get('PWD')+f"/itr-data-pipeline/data/external/OECM 20220504/{filename}.xlsx", sheet_name=sheet, dtype=str)
        df = df.applymap(lambda x: x.rstrip(), na_action='ignore')
        print(f"Region {region} Sector {subsector}")
        df = process_sector_benchmark (df, region, subsector, sector_elements)
        # It's tempting to concatenate these DataFrames, but doing so wrecks the nice PintArrays created for Production and EI
        # So instead, build up the respective dictionaries with each dataframe we process
        
        # Production is not conditioned on scope--we shouldn't even need it!
        new_prod_bm = {
            scope: {
                "benchmarks": [
                    {
                        "sector": itr_dict[subsector],
                        "region": region,
                        "benchmark_metric": { "units": "dimensionless" },
                        "scenario name": "OECM 1.5 Degrees",
                        "release date": "2022",
                        "unit": "dimensionless",
                        "projections": [
                            {
                                "year": year,
                                "value": value
                            }
                            for year, value in zip(df.index, df.d_Production)
                        ]
                    }
                ]
            }
            for scope in ['S1S2']
        }
        merge_bm_dicts(production_bm, new_prod_bm)

        bm_ei_scopes = {
            scope: {
                "benchmarks": [
                    {
                        "sector": itr_dict[subsector],
                        "region": region,
                        "benchmark_metric": { "units": ei_unit },
                        "scenario name": "OECM 1.5 Degrees",
                        "release date": "2022",
                        "unit": ei_unit,
                        "projections": [
                            {
                                "year": year,
                                "value": value.m
                            }
                            for year, value in zip(df.index, df[f"EI_{scope}"])
                        ]
                    }
                ]
            }
            for scope in ['S1S2', 'S1S2S3']
        }

        if df.S3.sum().m:
            bm_ei_scopes['S3'] = {
                "benchmarks": [
                    {
                        "sector": itr_dict[subsector],
                        "region": region,
                        "benchmark_metric": { "units": ei_unit },
                        "scenario name": "OECM 1.5 Degrees",
                        "release date": "2022",
                        "unit": ei_unit,
                        "projections": [
                            {
                                "year": year,
                                "value": value.m
                            }
                            for year, value in zip(df.index, df.EI_S3)
                        ]
                    }
                ]
            }

        merge_bm_dicts(ei_bm, bm_ei_scopes)


Region Global Sector Materials / Steel
Region Europe Sector Materials / Steel
Region North America Sector Materials / Steel
Region Global Sector Power Utilities
Region Europe Sector Power Utilities
Region North America Sector Power Utilities
Region Global Sector Gas Utilities
Region Europe Sector Gas Utilities
Region North America Sector Gas Utilities
Region Global Sector Utilities
Region Europe Sector Utilities
Region North America Sector Utilities
Region Global Sector Energy Industry
Region Europe Sector Energy Industry
Region North America Sector Energy Industry
Region Global Sector Road: LDV / Passenger Transport


,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
2,Subsector,Unit,NaN,2019,2025,2030,2035,2040,2045,2050,NaN,NaN,NaN
72,Road Transport (excluding vehicle manufacturing),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN
73,Road Transport - Scope 1:,[Mt CO2e],equals [Mt CO2],180.9938165735467,111.4040852347164,69.66889884332922,43.70734236216623,24.953554185471912,12.073295775667825,0,X,NaN,NaN
75,Road Transport - Scope 2:,[Mt CO2e],equals [Mt CO2],34.04364547238278,157.7915682000634,119.66035199369203,287.5489722052102,137.64468362393723,42.989691418338985,0,X,NaN,NaN
77,Road Transport - Scope 3:,0,equals [Mt CO2],6061.76415569142,5044.305525869982,4002.222831424446,1183.4554355885957,540.2694354845062,56.940371905846845,0,NaN,NaN,NaN
80,Road Transport - Scope 1:,[Mt CO2e],equals [Mt CO2],6242.757972264966,5155.7096111046985,4071.8917302677755,1227.162777950762,565.2229896699781,69.01366768151468,0,NaN,NaN,NaN
82,Road Transport - Scope 2:,[Mt CO2e],equals [Mt CO2],34.04364547238278,157.7915682000634,119.66035199369203,287.5489722052102,137.64468362393723,42.989691418338985,0,NaN,NaN,NaN
84,Road Transport - Scope 3:,[Mt CO2e],equals [Mt CO2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


/Users/michael/opt/miniconda3/envs/itr_env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


Region Europe Sector Road: LDV / Passenger Transport


,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
2,Subsector,Unit,NaN,2017,2018,2019,2020,2025,2030,2035,2040,2045,2050
72,Road Transport (excluding vehicle manufacturing),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,Road Transport - Scope 1:,[Mt CO2e],equals [Mt CO2],21.431514237550516,21.62008071287942,19.774754777662814,18.373847068564118,15.738022999032712,9.20914618691058,5.567913029841109,3.323148147487334,1.3762700791463853,0
75,Road Transport - Scope 2:,[Mt CO2e],equals [Mt CO2],0.557529759894157,0.7645672665186845,1.022334301142228,0.9295430122896454,39.76332884023827,48.0245583294453,40.506337315999545,16.860953637954957,4.514005077491844,0
77,Road Transport - Scope 3:,0,equals [Mt CO2],905.575581096,901.2669722958001,904.4612779607999,833.2015817118689,628.2086905414915,152.89436764916903,40.910515886494565,5.846325482422593,0,0
80,Road Transport - Scope 1:,[Mt CO2e],equals [Mt CO2],927.0070953335505,922.8870530086795,924.2360327384627,851.5754287804331,643.9467135405242,162.1035138360796,46.47842891633567,9.169473629909927,1.3762700791463853,0
82,Road Transport - Scope 2:,[Mt CO2e],equals [Mt CO2],0.557529759894157,0.7645672665186845,1.022334301142228,0.9295430122896454,39.76332884023827,48.0245583294453,40.506337315999545,16.860953637954957,4.514005077491844,0
84,Road Transport - Scope 3:,[Mt CO2e],equals [Mt CO2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


...

/Users/michael/opt/miniconda3/envs/itr_env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


Region North America Sector Road: Trucks / Freight Transport


,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
2,Subsector,Unit,NaN,2019,2025,2030,2035,2040,2045,2050,NaN,NaN,NaN
72,Road Transport (excluding vehicle manufacturing),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,Road Transport - Scope 1:,[Mt CO2e],equals [Mt CO2],22.360406710172665,15.17923741233191,5.383964247861818,4.596840539412637,1.7644438139501126,0.9439599890674362,0.3247185798971315,NaN,NaN,NaN
75,Road Transport - Scope 2:,[Mt CO2e],equals [Mt CO2],3.2095108255082883,28.746565709402145,42.108048622202574,54.982381055832846,19.340129786027184,10.909245010517383,8.687378820710645,NaN,NaN,NaN
77,Road Transport - Scope 3:,0,equals [Mt CO2],1769.1716438121,1196.0723830001373,317.67536398473067,77.49962247543004,8.931385147380471,0,0,NaN,NaN,NaN
80,Road Transport - Scope 1:,[Mt CO2e],equals [Mt CO2],1791.5320505222726,1211.2516204124693,323.05932823259246,82.09646301484268,10.695828961330584,0.9439599890674362,0.3247185798971315,NaN,NaN,NaN
82,Road Transport - Scope 2:,[Mt CO2e],equals [Mt CO2],3.2095108255082883,28.746565709402145,42.108048622202574,54.982381055832846,19.340129786027184,10.909245010517383,8.687378820710645,NaN,NaN,NaN
84,Road Transport - Scope 3:,[Mt CO2e],equals [Mt CO2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


/Users/michael/opt/miniconda3/envs/itr_env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


Region Global Sector Aluminium Industry
Region Europe Sector Aluminium Industry
Region North America Sector Aluminium Industry
Region Global Sector Materials / Cement
Region Europe Sector Materials / Cement
Region North America Sector Materials / Cement
Region Global Sector Buildings
Region Europe Sector Buildings
Region North America Sector Buildings
Region Global Sector Chemical Industry
Region Europe Sector Chemical Industry
Region North America Sector Chemical Industry
Region Global Sector Textile & Leather
Region Europe Sector Textile & Leather
Region North America Sector Textile & Leather


### Emit Sector Benchmark Data

In [12]:
# https://til.simonwillison.net/python/json-floating-point
def round_floats(o):
    if isinstance(o, float):
        return round(o, 4)
    if isinstance(o, dict):
        return {k: round_floats(v) for k, v in o.items()}
    if isinstance(o, (list, tuple)):
        return [round_floats(x) for x in o]
    return o

with open('production_bm.json', 'w') as f:
    json.dump(round_floats(production_bm), sort_keys=False, indent=2, fp=f)

with open('ei_bm.json', 'w') as f:
    json.dump(round_floats(ei_bm), sort_keys=False, indent=2, fp=f)